### Connect to Drive and importing dependencies

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
pip install swifter

In [3]:
import pandas as pd
import numpy as np
import re
import swifter
from tqdm import tqdm
import math
import statistics

### Read the data

In [4]:
pathTrain = "/content/drive/MyDrive/NLP Study Group/Problem 1/train.txt"
pathTest = "/content/drive/MyDrive/NLP Study Group/Problem 1/test.txt"

In [5]:
train = pd.read_csv(pathTrain, names=['text'], sep = "\n")
test = pd.read_csv(pathTest, names=['text'],  sep = "\n")
display(train)
display(test)

,text
0,Natural language processing -LRB- NLP -RRB- is...
1,"Specifically , it is the process of a computer..."
2,"In theory , natural language processing is a v..."
3,Natural language understanding is sometimes re...
4,"Whether NLP is distinct from , or identical to..."
...,...
1296,When punctuation and similar clues are not con...
1297,Effective natural language processing systems ...
1298,"As an example , processing text used in medica..."
1299,The process of developing text segmentation to...


,text
0,"In computational linguistics , word-sense disa..."
1,The solution to this problem impacts other com...
2,Research has progressed steadily to the point ...
3,A rich variety of techniques have been researc...
4,"Among these , supervised learning approaches h..."
...,...
166,WordNet is the most popular example of sense i...
167,The reason for adopting the HECTOR database du...
168,Evaluation measures .
169,During the evaluation of WSD systems two main ...


### N-Gram Model Class

In [10]:
class nGramModel:
  def __init__(self):
    self._n = None
    self._data = []
    self._selfData = []
    self._smoothingType = None
    
    self._countMatrix = []
    self._nmin1gramCounter = []
    self._ngramCounter = []
    self._nmin1gramCounterIndexer = []
    self._ngramCounterIndexer = []

    self._addk = None
    
  def tokenize(self, sentence):
    return sentence.split()

  def initiateMatrix(self, xAxis, yAxis):
    self._countMatrix.append([np.zeros((len(xAxis), len(yAxis)), dtype=float)])
    
  def endMarker(self, sentence):
    return sentence + str(" </S>")

  def nGramCounter(self, pandasSeriesOfSentence, n):
    words = (pandasSeriesOfSentence.str.split(' ').explode())
    tempGram = (words)
    for i in range(n-1):
      nextWord = words.groupby(level=0).shift(i*-1-1)
      tempGram = (tempGram + " " + nextWord)

    tempGram.dropna()
    return tempGram.value_counts()[:]

  def entropy(self, testData):
    listEntropy = []
    xContext = None
    yGram = None

    self._testData = testData.copy().apply(self.endMarker)
    for gram in range(self._n):
      if gram != 0: self._testData = '<S> ' + self._testData

    for sentence in self._testData:
      key = []
      sumProb = 0
      tokenized = sentence.split()
      
      if gram > 0:
        for i in range(len(tokenized)):
          key.append(tokenized[i])
          if len(key) == gram + 1:
            xgram = " ".join(key[:gram])
            ygram = " ".join(key[:])
            
            if xgram in self._nmin1gramCounterIndexer[gram][0] and ygram in self._ngramCounterIndexer[gram][0]:
              proba = self._countMatrix[gram][0][self._nmin1gramCounterIndexer[gram][0].index(xgram)][self._ngramCounterIndexer[gram][0].index(ygram)]
              sumProb = sumProb + proba * math.log2(proba)
            else: 
              proba = 0.75/len(self._nmin1gramCounterIndexer[gram][0])
              sumProb = sumProb + proba * math.log2(proba)
            key.pop(0)
      else:
        for i in range(len(tokenized)):
          if tokenized[i] in self._countMatrix[0][0]:
            proba = self._countMatrix[0][0][tokenized[i]]
            sumProb = sumProb + proba * math.log2(proba)  
          else: 
            proba = 0.75/len(self._countMatrix[0][0])
            sumProb = sumProb + proba * math.log2(proba)     
      
      listEntropy.append(-sumProb)

    return statistics.mean(listEntropy)

  def fit(self, x, n, smoothingType = 'non', addk = 0):
    self._n = n
    self._data = x.copy().swifter.apply(self.endMarker)
    self._smoothingType  = smoothingType
    self._addk = addk

    for gram in range(n):
      if gram == 0:
        self._ngramCounter.append([self.nGramCounter(self._data, gram+1)])
        self._nmin1gramCounter.append([])
        self._nmin1gramCounterIndexer.append([])
        self._ngramCounterIndexer.append([])
        self._countMatrix.append([self._ngramCounter[gram][0] / len(self._ngramCounter[gram][0])])
      else:
        self._data = '<S> ' + self._data
        self._nmin1gramCounter.append([self.nGramCounter(self._data, gram)])
        self._ngramCounter.append([self.nGramCounter(self._data, gram+1)])
        self._nmin1gramCounterIndexer.append([self._nmin1gramCounter[gram][0].index.to_list()])
        self._ngramCounterIndexer.append([self._ngramCounter[gram][0].index.to_list()])
        self.initiateMatrix(self._nmin1gramCounter[gram][0], self._ngramCounter[gram][0])

        for sentence in self._data:
          key = []

          tokenized = sentence.split()
          for i in range(len(tokenized)):
            key.append(tokenized[i])
            if len(key) == gram + 1:
              xgram = " ".join(key[:gram])
              ygram = " ".join(key[:])
              self._countMatrix[gram][0][self._nmin1gramCounterIndexer[gram][0].index(xgram)][self._ngramCounterIndexer[gram][0].index(ygram)] += 1
              key.pop(0)

        if self._smoothingType == 'non':
          for i in range(len(self._countMatrix[gram][0])):
            self._countMatrix[gram][0][i] = self._countMatrix[gram][0][i] / self._nmin1gramCounter[gram][0][i]
        elif self._smoothingType == 'laplace':
          for i in range(len(self._countMatrix[gram][0])):
            self._countMatrix[gram][0][i] = (self._countMatrix[gram][0][i] + 1) / (self._nmin1gramCounter[gram][0][i] + len(self._nmin1gramCounter[gram][0]))
        elif self._smoothingType == 'add-k':
          for i in range(len(self._countMatrix[gram][0])):
            self._countMatrix[gram][0][i] = (self._countMatrix[gram][0][i] + self._addk) / (self._nmin1gramCounter[gram][0][i] + (self._addk * len(self._nmin1gramCounter[gram][0])))

    if self._smoothingType == 'kneser-ney':
      for i in tqdm(range(len(self._countMatrix[self._n-1][0]))): #->74
        for j in range(len(self._countMatrix[self._n-1][0][i])): #->79
          context = self._ngramCounterIndexer[self._n-1][0][j]
          string = self._nmin1gramCounterIndexer[self._n-1][0][i]
          if context.startswith(string):
            pContinuationNominator = 0
            pContinuationDenominator = 0
            d = 1
            backStep = self._n
            tokenContext = context.replace(string, '')
            #firstTerm = max(0, ) / 

            while pContinuationNominator == 0:
              if backStep != 0:
                backStep = backStep - 1
                if backStep != 0:
                  pContinuationNominator = [item.endswith(tokenContext) for item in self._ngramCounterIndexer[backStep][0]].count(True)
                  pContinuationDenominator = len(self._ngramCounterIndexer[backStep][0])
              else:
                pContinuationNominator = 1
                pContinuationDenominator = len(self._ngramCounter[0][0])

            if backStep != self._n-1: d = 0.75

            firstTerm = max(0, (self._countMatrix[self._n-1][0][i][j] - d))/ self._nmin1gramCounter[self._n-1][0][i]
            B = [item.startswith(string) for item in self._ngramCounterIndexer[self._n-1][0]]
            A = self._ngramCounter[self._n-1][0]
            lambdaLaterExpression =  sum(a for a,b in zip(A, B) if b)
            lamd = (d / self._nmin1gramCounter[self._n-1][0][i]) * lambdaLaterExpression
            pContinuation = pContinuationNominator / pContinuationDenominator
            self._countMatrix[self._n-1][0][i][j] = firstTerm + lamd * pContinuation  


### 1-gram

In [7]:
ng = nGramModel()
ng.fit(train.text, 1, smoothingType='non')
ng.entropy(test.text)

1-gram


4.825769440410396

In [8]:
ng = nGramModel()
ng.fit(train.text, 1, smoothingType='laplace')
ng.entropy(test.text)


1-gram


4.825769440410396

In [10]:
ng = nGramModel()
ng.fit(train.text, 1, smoothingType='add-k', addk=5)
ng.entropy(test.text)

1-gram


4.825769440410396

### 2-gram

In [13]:
ng = nGramModel()
ng.fit(train.text, 2, smoothingType='non')
ng.entropy(test.text)

1-gram
Processing 2-gram


2.6230528091478513

In [15]:
ng = nGramModel()
ng.fit(train.text, 2, smoothingType='laplace')
ng.entropy(test.text)


1-gram
Processing 2-gram


0.7593649354450983

In [16]:
ng = nGramModel()
ng.fit(train.text, 2, smoothingType='add-k', addk=5)
ng.entropy(test.text)

1-gram
Processing 2-gram


0.3284907998484869

In [11]:
ng = nGramModel()
ng.fit(train.text, 2, smoothingType='kneser-ney')
ng.entropy(test.text)

100%|██████████| 5235/5235 [08:55<00:00,  9.78it/s]


2.429837788943742

### 3-gram

In [30]:
ng = nGramModel()
ng.fit(train.text, 3, smoothingType='non')
ng.entropy(test.text)

0.8718609481693802

In [31]:
ng = nGramModel()
ng.fit(train.text, 3, smoothingType='laplace')
ng.entropy(test.text)

0.03855510563005131

In [7]:
ng = nGramModel()
ng.fit(train.text, 3, smoothingType='add-k', addk=5)
ng.entropy(test.text)

0.020822585357205863

In [8]:
ng = nGramModel()
ng.fit(train.text, 3, smoothingType='kneser-ney')
ng.entropy(test.text)

100%|██████████| 21515/21515 [17:41<00:00, 20.27it/s]


0.8718609481693802